# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [ ]:
!pip install Levenshtein
!pip install weighted-levenshtein
!pip install qwerty-weighted-levenshtein
!pip install typo
import re
import typo
from Levenshtein import distance as LDist
from qwerty_weighted_levenshtein import qwerty_weighted_levenshtein as qDist
from tqdm import tqdm
from math import log
from random import choice, sample
from collections import Counter

In [97]:
# Helper functions & reading the data

# Text preprocessing function
def preprocess_text(string):
    string = string.lower() # lowercases
    string = re.sub(r"[^\w\d'\s]+",' ',string) # removes punctuation and special characters
    string = re.sub(r"[0-9]", "", string) # removes digits
    string = ' '.join(string.split()) # removes accidental spaces
    return string


def expand(x, k=None):
    if type(x) in [list, set]:
        [print(item) for item in (x if k is None else x[:k])]
    if type(x) == dict:
        [print(k, v) for k, v in (x.items() if k is None else list(x.items()[:k]))]


# Opens and reads the file with data
with open("/big.txt", "r") as f:
    text_full = f.readlines()
    text_full = [preprocess_text(x[:-1]) for x in text_full]
    text_full = [x for x in text_full if 50 >= len(x.split()) >= 5]

# Train/test random split
test_idx = sample(range(len(text_full)), 58)
train_idx = list(set(range(len(text_full))) - set(test_idx))
text_test = [text_full[i] for i in test_idx]
text_train = [text_full[i] for i in train_idx]

FileNotFoundError: [Errno 2] No such file or directory: '/content/big.txt'

In [106]:
def get_context(string, center, rng):
    """
    This is a helper function that takes a list and in index
    and returns a list of this index's word's neighbors (aka context)
    """
    return string[max(center-rng,0):center] + string[center+1:min(center+rng+1, len(string))]


def build_wwm(string_list, context_len=3):
    """
    Helper function; takes a list of strings
    and returns vocabulary and word context matrix
    """
    wwm = dict()   # word context matrix
    vocab = dict() # vocabulary count
    cont = dict()  # context count
    total = 0  # total n words count

    for sent in string_list:
        # processes each word in string
        sent = preprocess_text(sent).split()
        for i in range(len(sent)):
            # gets the word's context
            context = get_context(sent, i, context_len)
            for word in context:
                # updates context matrix
                temp = wwm.get(sent[i], dict())
                temp[word] = temp.get(word, 10) + 1
                wwm[sent[i]] = temp

                cont[word] = cont.get(word, 10) + 1
                total += 1
            vocab[sent[i]] = vocab.get(sent[i], 10) + 1

    # calculates PPMI for each value
    for k, v in wwm.items():
        for k1, v1 in v.items():
            v[k1] = max(0, log((v1*total)/(vocab[k]*cont[k1]), 2))
        wwm[k] = v

    return wwm, vocab, cont


wwm, vocab, context = build_wwm(text_train)

def get_candidates(word, m, k):
    """
    Helper function. Takes a word and word matrix
    and returns k most similar words
    """
    candidates, dist_thres = list(), float("inf")
    for item in m.keys():
        dist = qDist(word, item)
        if dist <= dist_thres:
            candidates.append([item, dist])
            dist_thres = min(dist_thres, max(3, dist))

    candidates.sort(key=lambda l:l[1])
    return candidates[:k]


def predict(word, word_candidates, context_replaced, wwm, voc):
    """
    Helper function -- takes a word, its candidates, context, word and context matricies
    and makes prediction for a single word
    """
    candidates = list()
    for i, candidate in enumerate(word_candidates):
        count = 1
        for context_word in context_replaced:
            count += wwm[candidate[0]].get(context_word[0], 0)
        candidates.append([candidate[0], count/(4**(candidate[1]-1.7))])

    return max(candidates, key=lambda l:l[1])[0]


def autocorrect(sentence, wwm=wwm, voc=vocab):
    """
    Top-level function for string spelling correction.
    It takes a string and returns a transformed version of it with mistakes corrected.

    Parameters:
    sentence (str): Text that needs to be corrected
    wwm (dict): A word-context matrix (do not touch it; it is set by default)
    voc (dict): Vocabulary matrix (do not touch it too)

    Returns:
    (str): corrected version of the provided text.

    """
    sentence = preprocess_text(sentence).split()
    # Gets candidates for all words in the sentence
    all_candidates = [get_candidates(x, wwm, 30) for x in sentence]
    # Gets the context version of the text
    sentence_substitutes = [x[0] for x in all_candidates]

    # Makes a prediction for each word and adds it to the new sentence
    new_sentence = list()
    for i in range(len(sentence)):
        new_sentence.append(predict(sentence[i], all_candidates[i], get_context(sentence_substitutes, i, 2), wwm, voc))

    # Joins the resulting list and returns
    return ' '.join(new_sentence)

## Justify your decisions

I used Norvig's 'big.txt' file to generate my own dataset for this problem.

The algorithm works as follows: it analyses each word and its context (by default, the context is 2 word before and 2 words after the given word). As a result, I have a dictionary of words and its neighboring words. When performing autocorrection, each words is given a list of candidates, and then the best candidate is chosen based on its context in the sentence and the learnt context words.

Norvig's and my algorithms are very similar because the logic behind them is very similar -- they find the most similar and frequent word from the known vocabulary and replace "unseen" words with "correct" ones. The idea of my improvement is to make this process more flexible by adding the ability to also consider neighboring words when making this decision (as opposed to simply choosing the most common word).

This way I can achieve several goals:
- Better robustness. Although the technique I used is not exactly a classic n-gram model, it allows for more flexibility in case of unseen "n-grams" and does a better work generalizing and analyzing the context and meanings.
- Context-sensitivity. The model learns the context of each word, and when making predictions, it chooses the candidate with the best context match.
- A potential ability to change correct but misused words. Since the model takes the context into account, it may change a misused word into a more appropriate but similar word, for example compare 'you"re' vs 'your'.

The candidates for each word are based on similarity -- Levenshtein distance (for evaluation) and Damerau-Levenshtein distance (for predicting), thus taking into account the mistakes associated with the keyboard layout.

One of the biggest downsides is this algorithm doesn't account for missing and extra spaces. Due to this assignment's limitation it is impossible to detect such cases, and iterating through each character and space to test for them is too time consuming.


As for evaluation, I take a valid sentence and make realistic looking mistakes: it randomly swaps two characters, replaces a character with its neighbour on a qwerty keyboard, adds/removes random characters (except for spaces) and so on. The number of such replacements is regulated with the noise parameter. Finally, it compares the noisy and corrected sentences with the initial valid one.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [95]:
# Examples of context sensitivity

print(autocorrect("dyad silk dress"))
print(autocorrect("He is already dyad"), "\n")

print(autocorrect("Are you dking sports"))
print(autocorrect("He is dking of fright!")) # there's no word 'species' in big.txt (:

dyed silk dress
he is already dead 

are you doing sports
he is dying of fright


In [108]:
class Evaluator():
    """
    This class provides an interface for evaluating the model performance.
    It takes a list of strings, applies some noise, calls the correction function,
    and evaluates the result using Levenshtein distance

    Parameters:
    text_data (List[str]): A collection of texts to perform tests on
    corrector (Callable): Your spelling correction function
    show_examples (Bool): If true, shows 5 examples of the corrector's output;
                          if false, it evaluates the performance of the entire text_data
                          without showing any text results of the corrector
    noise (float): Shows how many mistakes to generate. 0 means no mistakes, and the bigger the value, the more mistakes
    noisy_data (list): Kostyl'; used for fair comparison between algorithms (do not touch it !!)
    """
    def __init__(self, text_data, corrector, show_examples, noise, noisy_data=None):
        self.data = text_data
        self.corrector = corrector
        self.show_examples = show_examples
        self.noise = noise

        # cleaning text
        self.data = [preprocess_text(x) for x in self.data]
        # generating text with typos
        self.noisy_data = [self.create_errors(x) for x in self.data] if noisy_data is None else noisy_data
        # evaluation
        self.evaluate()


    def create_errors(self, string):
        """
        Helper function; adds realistic-looking mistakes to a given string
        """
        errer = typo.StrErrer(string)
        for i in range(int(len(string)*self.noise)):
            fun = choice([errer.missing_char, errer.extra_char, errer.nearby_char,
                          errer.similar_char, errer.repeated_char, errer.unichar])
            string = fun().result #very fun result
        return string


    def evaluate(self):
        """
        Main function that calls after this class's init function.
        Applies the corrector function to sentences with mistakes and prints the report
        """
        # if show_example is True, then it just prints 5 lines of sentences
        if self.show_examples == True:
            for i in range(min(5, len(self.data))):
                sent = self.data[i]
                noisy_sent = self.noisy_data[i]
                corrected_sent = self.corrector(noisy_sent)

                # Calculates the Levenshtein distances between correct & noisy and correct & corrected sentences
                dist_init = LDist(sent, noisy_sent)
                dist_corrected = LDist(sent, corrected_sent)

                print(f"Sent #{i+1} --- Length: {len(sent)}  # Mistakes initially: {dist_init}   # Mistakes in corrected: {dist_corrected}")
                print("    TRUTH", sent)
                print("    NOISY", noisy_sent)
                print("CORRECTED", corrected_sent)
                print()


        # if show_example is False, then it evaluates the entire dataset
        if self.show_examples == False:
            # Some values for statistics
            mistakes_init_count, mistakes_count, total_length_count = 0, 0, 0

            # Initilaizes a progress bar
            pbar = tqdm(total=len(self.data), position=0, leave=True)
            pbar.set_description_str("EVALUATING")

            for i in range(len(self.data)):
                sent = self.data[i]
                noisy_sent = self.noisy_data[i]
                corrected_sent = self.corrector(noisy_sent)

                # Calculates the Levenshtein distances between correct & noisy and correct & corrected sentences
                dist_init = LDist(sent, noisy_sent)
                dist_corrected = LDist(sent, corrected_sent)

                # Updates the counters
                total_length_count += len(sent)
                mistakes_init_count += dist_init
                mistakes_count += dist_corrected

                # Updates the progress bar
                pbar.update(1)
                pbar.set_postfix_str(f"Noisy sentence accuracy: {round(100-100*mistakes_init_count/total_length_count, 3)}%   Corrected sentence accuracy: {round(100-100*mistakes_count/total_length_count, 3)}%")

In [62]:
# Implements the Norvig's algorithm (with slight changes) (for comparison)

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(' '.join(text_train)))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [63]:
# Comparing the output of my and Norvig's solutions

print("EXAMPLES OF MY SOLUTION")
noisy = Evaluator(text_test, autocorrect, True, noise=0.1).noisy_data

print("\n\n\nEXAMPLES OF NORVIG'S SOLUTION")
norvig = lambda l: ' '.join(correction(x) for x in l.split())
Evaluator(text_test, norvig, True, 0.1, noisy)

EXAMPLES OF MY SOLUTION
Sent #1 --- Length: 63  # Mistakes initially: 4   # Mistakes in corrected: 0
    TRUTH updated editions will replace the previous one the old editions
    NOISY updated ediytions will replace the preuious one the old edtiohns
CORRECTED updated editions will replace the previous one the old editions

Sent #2 --- Length: 70  # Mistakes initially: 5   # Mistakes in corrected: 0
    TRUTH several varieties of whitlow are recognised but while it is convenient
    NOISY sevedral varicties of whitlow are recoggnised but whilc it is convenicnt
CORRECTED several varieties of whitlow are recognised but while it is convenient

Sent #3 --- Length: 64  # Mistakes initially: 6   # Mistakes in corrected: 1
    TRUTH realize the significance of his appearance before her now prince
    NOISY realize the significancce of his apprearwndce befire her now price
CORRECTED realize the significance of his appearance before her now price

Sent #4 --- Length: 65  # Mistakes initially: 6 

In [99]:
# COMPARISON

for noise in [0.05, 0.1, 0.15]:
    print("My solution, noise:", noise)
    noisy = Evaluator(text_test, autocorrect, False, noise).noisy_data

    print("\nNorvig's solution, noise:", noise)
    Evaluator(text_test, norvig, False, noise, noisy)
    print("\n\n\n")

My solution, noise: 0.05


EVALUATING: 100%|██████████| 58/58 [00:49<00:00,  1.18it/s, Noisy sentence accuracy: 96.256%   Corrected sentence accuracy: 98.743%]



Norvig's solution, noise: 0.05


EVALUATING: 100%|██████████| 58/58 [00:01<00:00, 50.49it/s, Noisy sentence accuracy: 96.256%   Corrected sentence accuracy: 98.771%]






My solution, noise: 0.1


...

EVALUATING: 100%|██████████| 58/58 [00:04<00:00, 14.50it/s, Noisy sentence accuracy: 93.155%   Corrected sentence accuracy: 97.15%]






My solution, noise: 0.15


EVALUATING: 100%|██████████| 58/58 [00:50<00:00,  1.16it/s, Noisy sentence accuracy: 90.053%   Corrected sentence accuracy: 96.032%]



Norvig's solution, noise: 0.15


EVALUATING: 100%|██████████| 58/58 [00:08<00:00,  6.77it/s, Noisy sentence accuracy: 90.053%   Corrected sentence accuracy: 95.613%]

Norvig's and my solution show similar results

If there are few mistakes, Norvig's algorithm tends to have better and faster results, but its performance and accuracy drops as sentences become more noisy.

For the case when noise=0.05, both algorithms import the accuracy by +2.5%; for moderately noisy sentences, by +4.1%. Finally, if there is a lot of mistakes, Norvig makes an improvement by +5.6% and I by +6%.

I may explain that in my algorithm time complexity depends on the number of words, so the time is the same in all cases, while Norvig has implemented a "bruteforce" solution to find the candidates, so more mistakes mean more candidates to choose from.

But both algorithms are essentially similar, because the main logic is to choose most common and "seen" words, and the idea of my improvement is to make this process a little more flexible by adding the notion of context.